In [0]:
    spark.conf.set("fs.azure.account.key.bdoifrssa.dfs.core.windows.net", "ZMTwlEStMs+cJyWhoY7uzBQsF98Ri3b7swSQKgZk+/WNAiyf+IvKN8H6huOWA8DhyK1IBTt9leYj+AStwDpz3A==")
    print("Test Connection and Display")

In [0]:
#df = spark.read.option("inferSchema","True").option("header", "True").option("delimiter", "\t").csv("abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/landing/Product.csv")
#display(df,10)

#df = spark.read.option("inferSchema","True").option("header", "True").option("delimiter", "\t").csv("abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/landing/Region.csv")
#display(df,10)

spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [0]:
print("Loading from Bronze Layer Starting!")

spark.sql(f"""DROP TABLE IF EXISTS bronze.SalespersonRegion""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Salesperson""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Sales""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Region""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Product""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Reseller""")
spark.sql(f"""DROP TABLE IF EXISTS bronze.Targets""")

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.SalespersonRegion
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/SalespersonRegion.csv';
            """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Salesperson
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Salesperson.csv';
            """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Sales
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Sales.csv';
            """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Region
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Region.csv';
        """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Product
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Product.csv';
        """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Reseller
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Reseller.csv';
        """)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.Targets
    USING CSV
    OPTIONS ('header'='true', sep '\t')
    LOCATION 'abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/bronze/Targets.csv';
        """)
    
print("Loading from Bronze Layer Complete!")

In [0]:
#df = spark.sql("SELECT * FROM targets WHERE to_number(Target, 'S$999,999,099') = 500000")
#display(df.printSchema())
#display(df,10)

#df.write \
#  .mode("overwrite") \
#  .option("path", f"abfss://bdo-ifrs@bdoifrssa.dfs.core.windows.net/silver/small_targets") \
#  .saveAsTable("small_targets")

df.write.format("delta").saveAsTable("silver.Targets")

In [0]:
df = spark.sql("SELECT * FROM bronze.targets")
df.write.format("delta").options('columnMapping.mode','name').saveAsTable("silver.targets")

df = spark.sql("SELECT * FROM bronze.product")
df.write.format("delta").saveAsTable("silver.product")

df = spark.sql("SELECT * FROM bronze.region")
df.write.format("delta").saveAsTable("silver.region")

df = spark.sql("SELECT * FROM bronze.reseller")
df.write.format("delta").saveAsTable("silver.reseller")

df = spark.sql("SELECT * FROM bronze.sales")
df.write.format("delta").saveAsTable("silver.sales")

df = spark.sql("SELECT * FROM bronze.salespersonregion")
df.write.format("delta").saveAsTable("silver.salespersonregion")

df = spark.sql("SELECT * FROM bronze.salesperson")
df.write.format("delta").saveAsTable("silver.salesperson")

In [0]:
from delta.tables import *

df = spark.sql("SELECT * FROM bronze.targets")
df.printSchema()

DeltaTable.createOrReplace(spark) \
  .tableName("silver.targets") \
  .addColumn("EmployeeID", "INT") \
  .addColumn("Target", "STRING") \
  .addColumn("TargetMonth", "STRING") \
  .execute()

df = spark.sql("INSERT INTO silver.targets SELECT * FROM bronze.targets")


In [0]:
%sql
DROP TABLE silver.targets_dlt;

CREATE TABLE silver.targets_dlt (
EmployeeID INT
,Target STRING
,TargetMonth STRING
) USING DELTA;

ALTER TABLE silver.targets_dlt ADD CONSTRAINT valid_target CHECK (to_number(Target, 'S$999,999,099') > 500000"));